# Welcome to the **stratosphere** dashboard!

***

In [15]:
from stratosphere import Stratosphere
from stratosphere import options
from IPython.display import HTML
from markdown import markdown


kb = Stratosphere(options.get("db.url_kb"))

count_entities = kb.db.pandas("SELECT count(*) as cnt FROM entities").cnt.iloc[0]
count_relationships = kb.db.pandas("SELECT count(*) as cnt FROM relationships").cnt.iloc[0]

mkd = f"""
The knowledge base contains {count_entities} entities and {count_relationships} relationships.
"""

HTML(markdown(mkd))

Web apps:

* [vk.com contacts explorer](http://127.0.0.1:8082/jupyter/voila/render/webapps/vk.com-contacts-explorer.ipynb): Explore previously seen vk.com contacts, highlighting their connections
* [Google sarch results](http://127.0.0.1:8082/jupyter/voila/render/webapps/google-search-results.ipynb): Review your past Google search results

Other resources:

* [Jupyter lab](/jupyter/lab): Create new Voilà web apps as notebooks, run ad-hoc analyses, and customize this page
* [Web apps directory](/jupyter/voila/tree/webapps): Access the complete list of web apps
* [SQLite web interface](/sqlite/): Experiment and query the underlying knowledge base with SQL

Recent system logs:


In [1]:
!tail -10 /tmp/supervisord.log

2023-04-13 14:48:56,640 INFO spawned: 'extractor' with pid 7
2023-04-13 14:48:56,647 INFO spawned: 'jupyter' with pid 8
2023-04-13 14:48:56,656 INFO spawned: 'mitmproxy' with pid 9
2023-04-13 14:48:56,663 INFO spawned: 'nginx' with pid 10
2023-04-13 14:48:56,690 INFO spawned: 'sqliteweb' with pid 12
2023-04-13 14:48:57,774 INFO success: extractor entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2023-04-13 14:48:57,774 INFO success: jupyter entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2023-04-13 14:48:57,774 INFO success: mitmproxy entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2023-04-13 14:48:57,774 INFO success: nginx entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2023-04-13 14:48:57,774 INFO success: sqliteweb entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
